# Análise Clima-Tempo

#### Semana 10 | Web Scraping, Regex e Pandas

## Objetivo do miniprojeto

A partir de sua escolha de uma cidade qualquer, faça a raspagem de uma página e descubra a temperatura média da próxima semana nesta cidade. Utilize o serviço [Tempo](https://www.tempo.com/) na sua solução. (Desejável incluir gráficos do Plotly).

## Projeto-piloto

Aqui, importamos as bibliotecas que utilizaremos para fazer a raspagem e análise de dados.

In [1]:
# Vai até o servidor e pega uma página HTML (ou outra)
import requests

# Manipula a página HTML
from bs4 import BeautifulSoup

# Faz a limpeza e mostra as informações
import pandas as pd

# Faz a manipulação numérica
import numpy as np

# Cria gŕaficos
import plotly.express as px

**Seleção do serviço de clima-tempo**

Apenas para a ilustração dos passos, vamos fazer a raspagem do [National Weather Service](https://forecast.weather.gov/), um serviço de climatologia norte-americano.


> **Para que estes passos sejam aplicáveis, é importante verificar se o serviço de clima-tempo apresenta as informações utilizando HTML ou Javascript. Estes passos só servem para respostas em HTML.**

In [2]:
# Localizamos uma cidade usando o próprio serviço de clima-tempo
url = "https://forecast.weather.gov/MapClick.php?lat=31.604&lon=-106.2511#.XoOR725v9PU"

Download da página. Uma resposta $200$ indica que o retorno da página foi bem-sucedido. 

In [3]:
# usamos a biblioteca requests para fazer o download da página web
page = requests.get( url )
page

<Response [200]>

A biblioteca `BeautifulSoup` permite que manipulemos o HTML da página. 

In [5]:
# criamos um objeto beautiful soup para poder fazer a raspagem dos dados
soup = BeautifulSoup(page.content, 'html.parser')
#soup
#print(soup.prettify())

Utilizando o modo de inspeção do navegador (ex: Chrome, Firefox, Safari...), identifique o elemento HTML em que deseja extrair as informações.

In [7]:
# após investigar, com um navegador web, a estrutura da página e localizar o elemento HTML que queremos, aplicamos o comando
seven_day = soup.find(id="seven-day-forecast")
#seven_day

Dentro do elemento HTML `seven-day-forecast`, pegamos todos os elementos da classe `tombstone-container`.

In [9]:
forecast_items = seven_day.find_all(class_="tombstone-container")
#forecast_items

O primeiro elemento da lista `forecast_items` representa a manhã de hoje.

In [10]:
morning = forecast_items[0]
morning

<div class="tombstone-container">
<p class="period-name">Today<br/><br/></p>
<p><img alt="Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. "/></p><p class="short-desc">Sunny</p><p class="temp temp-high">High: 95 °F</p></div>

In [11]:
# Impressão na tela formatado
print(morning.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 95 °F
 </p>
</div>


O segundo elemento da lista `forecast_items` representa o próximo período.

In [12]:
afternoon = forecast_items[1]
print(afternoon.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Partly cloudy, with a low around 62. West southwest wind 9 to 14 mph becoming east in the evening. Winds could gust as high as 21 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 62. West southwest wind 9 to 14 mph becoming east in the evening. Winds could gust as high as 21 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 62 °F
 </p>
</div>


**Extraindo informações da página**

A etiqueta representada pela variável `morning` contém toda a informação de que precisamos:

- O nome do item da previsão - neste caso, Morning

- A descrição das condições - está localizado no título da propriedade de img

- Uma descrição breve das condições - neste caso Patchy Blowing Dust then Cloudy

- A temperatura mínima - neste caso 81 °F

In [ ]:
morning

In [ ]:
#morning.find(class_="period-name").get_text()

In [13]:
period = morning.find(class_="period-name").get_text()
period

'Today'

In [14]:
short_desc = morning.find(class_="short-desc").get_text()
short_desc

'Sunny'

In [15]:
temp = morning.find(class_="temp").get_text()
temp

'High: 95 °F'

Uma opção seria pegar as informações contidas na tag `<img>`.

In [16]:
img = morning.find("img")
img

<img alt="Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. "/>

In [17]:
# Obtem o título (title) da descrição da img
desc = img['title']
desc

'Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. '

**Extraindo toda a informação da página**

Agora que sabemos como extrair a informação de um elemento, vamos obter todos os outros.

In [ ]:
seven_day

```html
<div class="tombstone-container">
    <p class="period-name">
        
    </p>
</div>
<div class="tombstone-container">
    <p class="period-name">
        
    </p>
</div>
<div class="tombstone-container">
    <p class="period-name">
        
    </p>
</div>
```

In [18]:
seven_day.select(".tombstone-container .period-name")

[<p class="period-name">Today<br/><br/></p>,
 <p class="period-name">Tonight<br/><br/></p>,
 <p class="period-name">Wednesday<br/><br/></p>,
 <p class="period-name">Wednesday<br/>Night</p>,
 <p class="period-name">Thursday<br/><br/></p>,
 <p class="period-name">Thursday<br/>Night</p>,
 <p class="period-name">Friday<br/><br/></p>,
 <p class="period-name">Friday<br/>Night</p>,
 <p class="period-name">Saturday<br/><br/></p>]

In [22]:
# Usa o seletor CSS para extrair todos os period-name dentro de tombstone-container
period_tags = seven_day.select(".tombstone-container .period-name")

# Usando list comprehensions (não vimos ainda)
# periods = [pt.get_text() for pt in period_tags]

periods = []
for pt in period_tags:
    periods.append(pt.get_text())
periods

['Today',
 'Tonight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday']

Vamos usar a mesma técnica para os outros três campos.

In [23]:
# Usando list comprehensions (não vimos ainda)
#short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]

short_descs_tags = seven_day.select(".tombstone-container .short-desc")
short_descs = []
for sd in short_descs_tags:
    short_descs.append(sd.get_text())
short_descs

['Sunny',
 'Partly Cloudy',
 'Sunny thenPatchyBlowing Dust',
 'Partly Cloudy',
 'Partly Sunny',
 'Clear',
 'Sunny',
 'Clear',
 'Sunny']

In [24]:
# Usando list comprehensions (não vimos ainda)
# temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]

temps_tags = seven_day.select(".tombstone-container .temp")
temps = [] 
for tp in temps_tags:
    temps.append(tp.get_text())
temps

['High: 95 °F',
 'Low: 62 °F',
 'High: 99 °F',
 'Low: 58 °F',
 'High: 92 °F',
 'Low: 53 °F',
 'High: 94 °F',
 'Low: 52 °F',
 'High: 96 °F']

In [25]:
# Usando list comprehensions (não vimos ainda)
# descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

descs_tags = seven_day.select(".tombstone-container img")
descs = []
for d in descs_tags:
    descs.append(d['title'])
descs

['Today: Sunny, with a high near 95. East southeast wind 7 to 15 mph becoming west northwest in the morning. Winds could gust as high as 22 mph. ',
 'Tonight: Partly cloudy, with a low around 62. West southwest wind 9 to 14 mph becoming east in the evening. Winds could gust as high as 21 mph. ',
 'Wednesday: Patchy blowing dust after noon. Sunny, with a high near 99. East southeast wind 7 to 12 mph becoming southwest 15 to 20 mph in the afternoon. Winds could gust as high as 30 mph. ',
 'Wednesday Night: Partly cloudy, with a low around 58. South southwest wind 13 to 18 mph decreasing to 6 to 11 mph after midnight. Winds could gust as high as 28 mph. ',
 'Thursday: Partly sunny, with a high near 92. West wind 6 to 13 mph. ',
 'Thursday Night: Clear, with a low around 53. West wind 6 to 11 mph becoming south after midnight. ',
 'Friday: Sunny, with a high near 94. West northwest wind 7 to 11 mph. ',
 'Friday Night: Clear, with a low around 52. West wind 7 to 11 mph. ',
 'Saturday: Sunny

#### Pandas

Um DataFrame é um objeto que pode armazenar dados tabulares, facilitando a análise. 

Vamos instanciar uma classe DataFrame e passar a lista de itens que temos. Vamos passar como parte de um dicionário. Cada chave do dicionário irá virar uma coluna no DataFrame e os elementos da lista irão se tornar os valores das colunas.

In [26]:
df_clima_tempo = pd.DataFrame({
    "Período": periods,
    "Minidescrição": short_descs,
    "Temperatura": temps,
    "Descrição":descs
})
df_clima_tempo

,Período,Minidescrição,Temperatura,Descrição
0,Today,Sunny,High: 95 °F,"Today: Sunny, with a high near 95. East southe..."
1,Tonight,Partly Cloudy,Low: 62 °F,"Tonight: Partly cloudy, with a low around 62. ..."
2,Wednesday,Sunny thenPatchyBlowing Dust,High: 99 °F,Wednesday: Patchy blowing dust after noon. Sun...
3,WednesdayNight,Partly Cloudy,Low: 58 °F,"Wednesday Night: Partly cloudy, with a low aro..."
4,Thursday,Partly Sunny,High: 92 °F,"Thursday: Partly sunny, with a high near 92. W..."
5,ThursdayNight,Clear,Low: 53 °F,"Thursday Night: Clear, with a low around 53. W..."
6,Friday,Sunny,High: 94 °F,"Friday: Sunny, with a high near 94. West north..."
7,FridayNight,Clear,Low: 52 °F,"Friday Night: Clear, with a low around 52. Wes..."
8,Saturday,Sunny,High: 96 °F,"Saturday: Sunny, with a high near 96. Southwes..."


Com a tabela anterior, podemos fazer a análise do clima em uma cidade, incluindo as brasileiras.

**Desafio**

Podemos utilizar [expressões regulares](https://pt.wikipedia.org/wiki/Express%C3%A3o_regular) e o método Series.str.extract para extrair o valor numérico das temperaturas. Esta [página](http://turing.com.br/material/regex/introducao.html) oferece uma introdução sobre o tópico.

In [ ]:
df_clima_tempo["Temperatura"]

Utilizamos o método [Series.str.extract](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html), que utiliza expressões regulares para extrair padrões do texto.

In [ ]:
# Neste caso, a expressão regular para extrair os digitos da string 
temp_nums = df_clima_tempo["Temperatura"].str.extract(r"(\d+)", expand=False)
temp_nums

In [ ]:
# O tipo resultante de Series.str.extract
type(temp_nums)

In [ ]:
# Converte os valores da coluna temp_num em inteiro
df_clima_tempo["Temperatura"] = temp_nums.astype('int')
df_clima_tempo["Temperatura"]

In [ ]:
df_clima_tempo

Calcula a média para as temperaturas altas e baixas. 

In [ ]:
df_clima_tempo["Temperatura"].mean()

### Novo Exemplo

In [ ]:
nova_url = "https://www.tempo.com/sao-paulo-proxima-semana.htm"
pag = requests.get(nova_url)
soup = BeautifulSoup(pag.content, 'html.parser')

In [ ]:
next_week = soup.find(class_="datos-dos-semanas")
print(next_week.prettify())

In [ ]:
pred = next_week.find(class_="prediccion")
print(pred.prettify())